In [ ]:
import preamble

# Constants

In [ ]:
import numpy as np
import time 

# Experiment
USER_ID = "11_P=0"

# Trajectory Planner
REFERENCE_TRAJECTORY_CSV_PATH = '../data/reference_path.csv'
INIT_POSE = np.array([-0.54, -0.27, 0.0 + 0.02, -1.21, -1.21, 1.25]) # Also used for Robot

# Physical PD
PHYSICAL_PD_SCALER = 4
K_p = 19 * PHYSICAL_PD_SCALER
B_p = 11 * PHYSICAL_PD_SCALER

# Physical Admittance
VARIABLE_ADMITTANCE_GAIN_HYPERPARAMETER = 0.02

# System dynamics
M = 2 * 2.0
B = 17.7

# Real time data collection
TABLE_COLUMN_NAMES = [
    't',
    'x',
    'x_dot',
    'F_h',
    'ref_going_forward',
    'x_ref',
    'x_dot_ref',
    'e',
    'e_dot',
    'P_hat',
    'V_hat',
    'c_p',
    'c_v',
    'verbal_P_tracking_term',
    'verbal_D_tracking_term',
    'verbal_PD_tracking_term',
    'A_v',
    'U_v',
    'physical_P_tracking_term',
    'physical_D_tracking_term',
    'physical_PD_tracking_term',
    'A_p',
    'U_p',
    'successful_utterance',
    'is_speaking',
    'phrase',
    'U',
    'human_disturbance',
    'system_dynamics_input',
]
DATA_COLLECTION_SAMPLING_RATE = 200

# Real time data plotting
REAL_TIME_DATA_PLOTTING_ENABLED = False
PLOTTING_SAMPLING_RATE = 30
DISPLAY_TIME_WINDOW = 5

USE_CAMERA = True

# Robot
ROBOT_IP = '169.254.9.43'
FORCE_EWMA_TAU = 0.5
TRANSLATIONAL_FORCE_DEADBAND = 2

# Experiment

This baseline is made up of all componentes: Physical PD, Physical Admittance, Verbal PD, Verbal Admittance, F2L, Compliance Estimator, and $J$ Optimization Method.

In [ ]:
from timer import Timer
from trajectory_planner import TrajectoryPlanner
from model import SVMKNNModel
import pickle
from vocalizer import Vocalizer
from virtual_dynamics import SimpleVirtualDynamics
from tabular_data_store import TabularDataStore
from plot_client import PlotClient
from robot import Robot
from console_command_thread import ConsoleCommandThread
import scipy.stats
from phrase_mapping import PHRASE_MAPPING, MEAN_WORDS_PER_PHRASE
from rgbd_stream import RGBDStream_iOS
from camera_feed import CameraFeed

#############
### SETUP ###
#############

### Experiment ###
experiment_timer = Timer()

### Trajectory Planner ###
traj = TrajectoryPlanner(REFERENCE_TRAJECTORY_CSV_PATH, INIT_POSE[:3])

### System dynamics ###
vd = SimpleVirtualDynamics(M=M, B=B, K=0)

### Real time data collection ###
table = TabularDataStore(column_names=TABLE_COLUMN_NAMES)
table_timer = Timer()

### Real time data plotting ###
if REAL_TIME_DATA_PLOTTING_ENABLED:
    plot = PlotClient()
    
    plot.create_plot('01_e', title='$e$ vs Time', xlabel='Time (s)', ylabel='Displacement (m)')
    plot.create_line("01_e", "e_x", color="r", label="$e_x$")
    plot.create_line("01_e", "e_y", color="g", label="$e_y$")
    plot.create_line("01_e", "e_z", color="b", label="$e_z$")

    plot.create_plot('02_e_norm', title='$||e||$ vs Time', xlabel='Time (s)', ylabel='Displacement (m)')
    plot.create_line("02_e_norm", "e_norm", label="$||e||$")

    plot.create_plot('03_e_log_norm', title='$\log||e||$ vs Time', xlabel='Time (s)', ylabel='Log Displacement ($\log$ m)')
    plot.create_line("03_e_log_norm", "e_log_norm", label="$\log||e||$")

    plot.create_plot('04_e_dot', title='$\dot{e}$ vs Time', xlabel='Time (s)', ylabel='Velocity (m/s)')
    plot.create_line("04_e_dot", "e_dot_x", color="r", label="$\dot{e}_x$")
    plot.create_line("04_e_dot", "e_dot_y", color="g", label="$\dot{e}_y$")
    plot.create_line("04_e_dot", "e_dot_z", color="b", label="$\dot{e}_z$")

    plot.create_plot('05_e_dot_norm', title='$||\dot{e}||$ vs Time', xlabel='Time (s)', ylabel='Velocity (m/s)')
    plot.create_line("05_e_dot_norm", "e_dot_norm", label="$||\dot{e}||$")

    plot.create_plot('06_e_dot_log_norm', title='$\log||\dot{e}||$ vs Time', xlabel='Time (s)', ylabel='Log Velocity ($\log$ m/s)')
    plot.create_line("06_e_dot_log_norm", "e_dot_log_norm", label="$\log||\dot{e}||$")

    plot.create_plot('07_compliance', title='Estimated Compliance vs Time', xlabel='Time (s)', ylabel='Compliance [0, 1]', ylim=[0,1])
    plot.create_line("07_compliance", "P_hat", color="orange", label="$\hat{P}$")
    plot.create_line("07_compliance", "V_hat", color="purple", label="$\hat{V}$")

    plot.create_plot("08_J_v_proj", title="Projected $J_v$ Profile", xlabel="Future Time (s)", ylabel="Impulse (Ns)")
    plot.create_line("08_J_v_proj", "J_v_x", color="r", label="$J_{v,x}$")
    plot.create_line("08_J_v_proj", "J_v_y", color="g", label="$J_{v,y}$")
    plot.create_line("08_J_v_proj", "J_v_z", color="b", label="$J_{v,z}$")

    plot.create_plot('09_U_v', title='$U_v$ vs Time', xlabel='Time (s)', ylabel='$U_v$ (N)')
    plot.create_line("09_U_v", "U_v_x", color="r", label="$U_{v,x}$")
    plot.create_line("09_U_v", "U_v_y", color="g", label="$U_{v,y}$")
    plot.create_line("09_U_v", "U_v_z", color="b", label="$U_{v,z}$")

    plot.create_plot('10_U_p', title='$U_p$ vs Time', xlabel='Time (s)', ylabel='$U_p$ (N)')
    plot.create_line("10_U_p", "U_p_x", color="r", label="$U_{p,x}$")
    plot.create_line("10_U_p", "U_p_y", color="g", label="$U_{p,y}$")
    plot.create_line("10_U_p", "U_p_z", color="b", label="$U_{p,z}$")

    plot.create_plot('11_U_norm', title='$||U||$ vs Time', xlabel='Time (s)', ylabel='$||U||$ (N)')
    plot.create_line("11_U_norm", "U_p_norm", color="orange", label="$||U_p||$")
    plot.create_line("11_U_norm", "U_v_norm", color="purple", label="$||U_v||$")

    plot_timer = Timer()

if USE_CAMERA:
    rgbd_stream = RGBDStream_iOS()
    calibration_matrix = np.load('calibration_matrix.npy')
    camera_feed = CameraFeed('Camera Feed', rgbd_stream, calibration_matrix)

    yellow_dot_timer = Timer()
    while yellow_dot_timer.t() < 7.0:
        camera_feed.draw_world_point(INIT_POSE[:3], radius=10, color=(0xff, 0xff, 0))
        camera_feed.update_window()

### Robot ###
with Robot(
    ROBOT_IP,
    default_axes=Robot.TRANSLATION,
    init_pose=INIT_POSE,
    force_ewma_tau=FORCE_EWMA_TAU,
    translational_force_deadband=TRANSLATIONAL_FORCE_DEADBAND,
) as r, ConsoleCommandThread() as c:

    ##################
    ### EXPERIMENT ###
    ##################

    experiment_timer.reset()

    while c.is_alive():
        period_start = r.control.initPeriod()

        ### Current state ###
        t = experiment_timer.t()
        dt = experiment_timer.dt()
        x = r.get_pose()
        x_dot = r.get_velocity()
        F_h = r.get_force()

        ### Trajectory Planner ###
        ref_going_forward = traj.update_reference_trajectory(x)
        x_ref, x_dot_ref = traj.get_closest_target(x)

        ### Tracking errors ###
        e = x_ref - x
        e_dot = x_dot_ref - x_dot

        ### Physical PD ###
        physical_P_tracking_term = K_p * e
        physical_D_tracking_term = B_p * e_dot
        physical_PD_tracking_term = physical_P_tracking_term + physical_D_tracking_term

        ### Physical Admittance ###
        A_p = 1 - np.exp(-VARIABLE_ADMITTANCE_GAIN_HYPERPARAMETER * np.linalg.norm(physical_PD_tracking_term - F_h))
        U_p = physical_PD_tracking_term - A_p*F_h

        ### System dynamics ###
        U = U_p + 0
        human_disturbance = F_h - 0
        system_dynamics_input = U + human_disturbance
        vd.apply_force(system_dynamics_input, dt)
        r.set_velocity(vd.get_velocity(), acceleration=10)

        ### Real time data collection ###
        if table_timer.t() >= DATA_COLLECTION_SAMPLING_RATE**(-1):
            table.append_row((
                t,
                x,
                x_dot,
                F_h,
                ref_going_forward,
                x_ref,
                x_dot_ref,
                e,
                e_dot,
                np.nan,
                np.nan,
                np.nan,
                np.nan,
                np.nan,
                np.nan,
                np.nan,
                np.nan,
                np.nan,
                physical_P_tracking_term,
                physical_D_tracking_term,
                physical_PD_tracking_term,
                A_p,
                U_p,
                np.nan,
                np.nan,
                np.nan,
                U,
                human_disturbance,
                system_dynamics_input,
            ))
            
            table_timer.reset()
            
        ### Real time data plotting ###
        if REAL_TIME_DATA_PLOTTING_ENABLED:
            if plot_timer.t() >= PLOTTING_SAMPLING_RATE**(-1):
                plot.update_line("01_e", "e_x", (t, e[0]))
                plot.update_line("01_e", "e_y", (t, e[1]))
                plot.update_line("01_e", "e_z", (t, e[2]))
                plot.config_plot("01_e", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("02_e_norm", "e_norm", (t, np.linalg.norm(e)))
                plot.config_plot("02_e_norm", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("03_e_log_norm", "e_log_norm", (t, np.log(np.linalg.norm(e))))
                plot.config_plot("03_e_log_norm", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("04_e_dot", "e_dot_x", (t, e_dot[0]))
                plot.update_line("04_e_dot", "e_dot_y", (t, e_dot[1]))
                plot.update_line("04_e_dot", "e_dot_z", (t, e_dot[2]))
                plot.config_plot("04_e_dot", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("05_e_dot_norm", "e_dot_norm", (t, np.linalg.norm(e_dot)))
                plot.config_plot("05_e_dot_norm", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("06_e_dot_log_norm", "e_dot_log_norm", (t, np.log(np.linalg.norm(e_dot))))
                plot.config_plot("06_e_dot_log_norm", xlim=(t - DISPLAY_TIME_WINDOW, t))

                # plot.update_line("07_compliance", "P_hat", (t, P_hat))
                # plot.update_line("07_compliance", "V_hat", (t, V_hat))
                plot.config_plot("07_compliance", xlim=(t - DISPLAY_TIME_WINDOW, t))

                # timesteps = np.arange(len(U_v_profile)) * dt_proj
                # plot.update_line("08_J_v_proj", "J_v_x", (timesteps, impulse_curve[:, 0]), mode="replace")
                # plot.update_line("08_J_v_proj", "J_v_y", (timesteps, impulse_curve[:, 1]), mode="replace")
                # plot.update_line("08_J_v_proj", "J_v_z", (timesteps, impulse_curve[:, 2]), mode="replace")
                # No sliding window for projection plot

                # plot.update_line("09_U_v", "U_v_x", (t, U_v[0]))
                # plot.update_line("09_U_v", "U_v_y", (t, U_v[1]))
                # plot.update_line("09_U_v", "U_v_z", (t, U_v[2]))
                plot.config_plot("09_U_v", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("10_U_p", "U_p_x", (t, U_p[0]))
                plot.update_line("10_U_p", "U_p_y", (t, U_p[1]))
                plot.update_line("10_U_p", "U_p_z", (t, U_p[2]))
                plot.config_plot("10_U_p", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot.update_line("11_U_norm", "U_p_norm", (t, np.linalg.norm(U_p)))
                # plot.update_line("11_U_norm", "U_v_norm", (t, np.linalg.norm(U_v)))
                plot.config_plot("11_U_norm", xlim=(t - DISPLAY_TIME_WINDOW, t))

                plot_timer.reset()
        
        if USE_CAMERA:
            p_r = x + U_p / (np.linalg.norm(U_p) + 1e-6) * 2.0 * 0.075 * np.log(np.linalg.norm(U_p) + 1.0)
            p_h = x + F_h / (np.linalg.norm(F_h) + 1e-6) * 2.0 * 0.075 * np.log(np.linalg.norm(F_h) + 1.0)
            camera_feed.draw_world_arrow(x - np.array([0.0, 0.0, 0.025]), p_r - np.array([0.0, 0.0, 0.025]), thickness=6, color=(0x3C / 0xFF, 0x91 / 0xFF, 0xE6 / 0xFF))
            camera_feed.draw_world_arrow(x, p_h, thickness=6, color=(0xFF / 0xFF, 0x87 / 0xFF, 0x1F / 0xFF))
            camera_feed.update_window()

        r.control.waitPeriod(period_start)

table.to_pandas().to_pickle(f'../data/experiments/physical_only/{USER_ID}_{time.time()}.pkl')